In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from scipy.stats import norm

import os
import time
import itertools

from glove.mse_model import *

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-tnydvcw0 because the default path (/home/jaron/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
# import file names
files = os.listdir("data/")
files = [f for f in files if "processed" in f]
files

['EXP0019_MS001_processed.csv',
 'EXP0019_DSM27147_processed.csv',
 'EXP0019_MS008_processed.csv',
 'EXP0019_MS014_processed.csv']

# fit gLV models

In [3]:
# for each data set 
for file in files:
    # import data
    df = pd.read_csv(f"data/{file}")

    # determine species names 
    species = df.columns.values[2:]
    n_species = len(species)

    # instantiate gLV fit 
    model = gLV(species, df) 

    # fit to data 
    model.fit()
    
    # plot parameter distribution
    Avec = model.params[n_species:]
    Aij_std = np.sqrt(np.diag(model.Ainv))[n_species:]
    
    plt.figure(figsize=(18,18))
    # set counter for parameter std. 
    k = 0

    for i in range(n_species):
        for j in range(n_species):
            plt.subplot(n_species, n_species, k+1)
            a = np.linspace(Avec[k]-np.std(Avec), Avec[k]+np.std(Avec))
            plt.plot(a, norm.pdf(a,Avec[k],Aij_std[k]))
            plt.axvline(x=0, c='k', alpha=.5)
            k += 1
            if j == 0:
                plt.ylabel(species[i], fontsize=18)
            if i == n_species-1:
                plt.xlabel(species[j], fontsize=18)
            #plt.xlim([-2,2])

    plt.suptitle(file.split("_")[1], fontsize=24)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig("params/"+file.split("_")[1]+".pdf")
    plt.close()
    
    # compute Wald test for each parameter
    std_errors = np.sqrt(np.diag(model.Ainv))
    walds = model.params/std_errors
    wald_p_vals = 2*norm.cdf(-np.abs(walds))

    # list of parameter names 
    param_names = []
    for s1 in species:
        for s2 in species:
            param_names += [s1+"*"+s2]
    param_names = list(species) + param_names
    
    # save to df 
    df = pd.DataFrame()
    df["Param name"] = param_names
    df["Param value"] = model.params
    df["Param stdv"]  = np.sqrt(np.diag(model.Ainv))
    df["Param p-value"] = wald_p_vals
    for j, param_name in enumerate(param_names):
        df[param_name]  = model.Ainv[:, j]
    df.to_csv("params/"+file.split("_")[1]+".csv", index=False)

Total samples: 63, Updated regularization: 1.00e-05
Loss: 17.595, Residuals: 0.049
Loss: 10.667, Residuals: 0.022
Loss: 7.690, Residuals: -0.056
Loss: 7.487, Residuals: -0.070
Loss: 7.111, Residuals: -0.060
Loss: 6.571, Residuals: -0.030
Loss: 6.104, Residuals: -0.032
Loss: 6.019, Residuals: -0.026
Loss: 5.964, Residuals: -0.021
Loss: 5.889, Residuals: -0.014
Loss: 5.876, Residuals: -0.016
Loss: 5.851, Residuals: -0.018
Loss: 5.807, Residuals: -0.021
Loss: 5.804, Residuals: -0.011
Loss: 5.777, Residuals: -0.015
Loss: 5.728, Residuals: -0.021
Loss: 5.728, Residuals: -0.021
Loss: 5.721, Residuals: -0.019
Loss: 5.711, Residuals: -0.018
Loss: 5.691, Residuals: -0.021
Loss: 5.691, Residuals: -0.021
Loss: 5.690, Residuals: -0.020
Loss: 5.665, Residuals: -0.021
Loss: 5.665, Residuals: -0.022
Loss: 5.664, Residuals: -0.021
Loss: 5.663, Residuals: -0.020
Loss: 5.655, Residuals: -0.019
Loss: 5.639, Residuals: -0.021
Loss: 5.639, Residuals: -0.021
Loss: 5.638, Residuals: -0.019
Loss: 5.613, Resid

Loss: 249.828, Residuals: -0.013
Loss: 249.827, Residuals: -0.013
Loss: 249.827, Residuals: -0.013
Loss: 249.827, Residuals: -0.013
Loss: 249.827, Residuals: -0.013
Loss: 249.827, Residuals: -0.013
Evidence 812.820
Pass count  1
Total samples: 63, Updated regularization: 1.00e-05
Loss: 15.632, Residuals: 0.012
Loss: 9.186, Residuals: -0.000
Loss: 7.413, Residuals: -0.017
Loss: 6.804, Residuals: -0.053
Loss: 6.509, Residuals: -0.029
Loss: 6.053, Residuals: -0.040
Loss: 5.537, Residuals: -0.043
Loss: 5.431, Residuals: -0.014
Loss: 5.251, Residuals: -0.010
Loss: 5.054, Residuals: -0.003
Loss: 5.036, Residuals: -0.006
Loss: 5.016, Residuals: -0.007
Loss: 4.980, Residuals: -0.011
Loss: 4.922, Residuals: -0.015
Loss: 4.921, Residuals: -0.016
Loss: 4.919, Residuals: -0.015
Loss: 4.916, Residuals: -0.011
Loss: 4.891, Residuals: -0.013
Loss: 4.846, Residuals: -0.013
Loss: 4.812, Residuals: -0.010
Loss: 4.750, Residuals: -0.012
Loss: 4.745, Residuals: 0.004
Loss: 4.693, Residuals: -0.002
Loss: 4

Loss: 247.947, Residuals: -0.011
Loss: 247.751, Residuals: -0.011
Loss: 247.736, Residuals: -0.011
Loss: 247.180, Residuals: -0.012
Loss: 247.176, Residuals: -0.012
Evidence 804.418
Updating hyper-parameters...
Total samples: 63, Updated regularization: 7.33e-01
Loss: 249.294, Residuals: -0.012
Evidence 805.396
Updating hyper-parameters...
Total samples: 63, Updated regularization: 7.40e-01
Loss: 250.158, Residuals: -0.012
Loss: 250.148, Residuals: -0.011
Loss: 249.757, Residuals: -0.012
Loss: 249.756, Residuals: -0.012
Evidence 806.025
Pass count  1
Total samples: 44, Updated regularization: 1.00e-05
Loss: 14.744, Residuals: -0.011
Loss: 7.431, Residuals: -0.029
Loss: 5.499, Residuals: -0.020
Loss: 4.648, Residuals: -0.010
Loss: 4.202, Residuals: 0.023
Loss: 4.171, Residuals: 0.030
Loss: 3.914, Residuals: -0.005
Loss: 3.890, Residuals: 0.008
Loss: 3.682, Residuals: -0.006
Loss: 3.394, Residuals: -0.033
Loss: 3.367, Residuals: -0.010
Loss: 3.319, Residuals: -0.019
Loss: 3.239, Residual

Loss: 171.159, Residuals: 0.013
Loss: 171.158, Residuals: 0.013
Loss: 171.156, Residuals: 0.013
Loss: 171.156, Residuals: 0.013
Loss: 171.156, Residuals: 0.013
Loss: 171.155, Residuals: 0.013
Loss: 171.154, Residuals: 0.013
Loss: 171.154, Residuals: 0.013
Loss: 171.154, Residuals: 0.013
Loss: 171.154, Residuals: 0.013
Loss: 171.154, Residuals: 0.013
Evidence 550.571
Updating hyper-parameters...
Total samples: 44, Updated regularization: 1.72e+00
Loss: 172.474, Residuals: 0.012
Loss: 172.417, Residuals: 0.012
Loss: 172.347, Residuals: 0.012
Loss: 172.300, Residuals: 0.011
Loss: 172.297, Residuals: 0.012
Loss: 172.291, Residuals: 0.012
Loss: 172.284, Residuals: 0.012
Loss: 172.284, Residuals: 0.012
Loss: 172.282, Residuals: 0.012
Loss: 172.281, Residuals: 0.012
Loss: 172.280, Residuals: 0.012
Loss: 172.280, Residuals: 0.012
Loss: 172.280, Residuals: 0.012
Loss: 172.279, Residuals: 0.012
Loss: 172.279, Residuals: 0.012
Loss: 172.279, Residuals: 0.012
Loss: 172.278, Residuals: 0.012
Loss: 